# Primer Intento de clasificador



Como el titulo lo dice, primer intento de clasificador usando como referencia el laboratorio x .
Se considera para este caso el archivo movies_final_imdb el cual contiene:
    
    movieId+nota_imbd_aprox+duracion+categoria+actores+directores
    
    
Nota : Falta agregar los directores y quizas las fechas para ve como cambia. 

In [39]:
## Librerias Basicas:
import csv
import numpy as np
import pandas as pd

#Metricas y Estadisticas:
from statsmodels.stats.weightstats import ttest_ind
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn import cross_validation


# Clasificadores : Tree, SVM, RandomForest, Regression, Naive Bayes,KNN......
from sklearn.svm import SVC  
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier



Cargamos los datasets.

In [40]:
movies = pd.read_csv("movies_final.csv", header = 0)
score = pd.read_csv("movielens.csv", header = 0)

movies_header = list(movies.columns.values)
movies = movies.as_matrix()

score = np.ravel(score.as_matrix())

[74  0  1 ...,  0  0  0]


# Reestructuracion del codigo y validación.





Esta nueva seccion calcula el promedio del accuracy de varios tipos de clasificadores y despues corre un test estadistico para determinar si existe una diferencia significativa entre el clasificador que se está probando y los otros.

~Algo asi como un cross validation.


Definimos las funciones del clasificador:

In [132]:
#Variar el num_test modifica la probabilidad de estar mas cerca del promedio al hacer un t-test. 
#Creo que con 35 es suficiente para ese tipo de distribuciones.

def run_classifier(clf, X, y, num_tests=35):
    scores = []
    
    for i in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test, y_test))

    return np.array(scores)

Agregamos varios tipos de clasificadores :

In [133]:
c0 = ("Base", DummyClassifier(strategy='stratified'))
c1 = ("SVM", SVC(kernel='linear'))
c2 = ("DT", DecisionTreeClassifier())
c3 = ("NB", GaussianNB())
c4 = ("KNN", KNeighborsClassifier(n_neighbors=5))
c5 = ("RandomForest", RandomForestClassifier(n_estimators=10))

classifiers = [c0, c1, c2, c3, c4,c5,]
result_list = []

Corremos las pruebas para cada test. 

In [134]:
#Funcion Pesada, correr solo una vez. 

for name, clf in classifiers:
    accuracys = run_classifier(clf ,movies , score)
    result_list.append((name, accuracys))
    

In [135]:
#Impresion de los resultados

print("+ indica diferencia significativa\n")

for name1, results1 in result_list:
    print("Comparando %s - Accuracy: %.2f" % (name1, results1.mean()))
    for name2, results2 in result_list:
        if name1 == name2:
            continue

        _, p_value, __ = ttest_ind(results1, results2)
        
        if p_value <= 0.05:
            sig = "+"
        else:
            sig = ""
        print("%s:\t%.2f %s" % (name2, results2.mean(), sig))
    print()

+ indica diferencia significativa

Comparando Base - Accuracy: 0.45
SVM:	0.61 +
DT:	0.54 +
NB:	0.38 +
KNN:	0.54 +
RandomForest:	0.57 +
()
Comparando SVM - Accuracy: 0.61
Base:	0.45 +
DT:	0.54 +
NB:	0.38 +
KNN:	0.54 +
RandomForest:	0.57 +
()
Comparando DT - Accuracy: 0.54
Base:	0.45 +
SVM:	0.61 +
NB:	0.38 +
KNN:	0.54 
RandomForest:	0.57 +
()
Comparando NB - Accuracy: 0.38
Base:	0.45 +
SVM:	0.61 +
DT:	0.54 +
KNN:	0.54 +
RandomForest:	0.57 +
()
Comparando KNN - Accuracy: 0.54
Base:	0.45 +
SVM:	0.61 +
DT:	0.54 
NB:	0.38 +
RandomForest:	0.57 +
()
Comparando RandomForest - Accuracy: 0.57
Base:	0.45 +
SVM:	0.61 +
DT:	0.54 +
NB:	0.38 +
KNN:	0.54 +
()


# Predicciones de algunas peliculas

Usando la siguiente funcion, se puede imprimir mas facilmente los resultados y luego calcular algunas metricas importantes. 

In [136]:
for clf_name, clf in classifiers:
    X_train, X_test, y_train, y_test = train_test_split(movies, score, test_size =  0.50)
    clf.fit(X_train, y_train)
        

In [138]:
sample_movies = [("Toy Story", movies[0], score[0]),("Avatar", movies[3111], score[3111]),
                 ("Titanic",movies[458],score[458]),("Transformers",movies[4057],score[4057]),
                 ("Never Say Never",movies[4189],score[4189])]
predicted_scores = []

# Toma un vector de tuplas nombre,clasificador y otro de nombre_pelicula,atributos,score_original y
# trata de predecir el score de esto para luego compararlos.
def predict_scores(clf, sample_movies):    
    for clf_name , clf in clf:
        for sample_name, sample_data, sample_score in sample_movies:
            predicted_scores.append((clf_name, sample_name,clf.predict(sample_data),sample_score))
    return predicted_scores

res = predict_scores(classifiers, sample_movies)

def predict_scores_toString(predicted_scores):
    string = "Clf\t\t\tNombrePelicula\t\t\tNotaPredicha\t\tNotaReal\n"
    for item in predicted_scores:
        for elements in item:
            string = str(string) + str(elements) + "\t\t\t"
        string = str(string) + "\n"
    return string
        
#Muchos Warnings :(        
        
res2 = predict_scores_toString(res)
print res2

Clf			NombrePelicula			NotaPredicha		NotaReal
Base			Toy Story			[3]			4			
Base			Avatar			[3]			4			
Base			Titanic			[2]			3			
Base			Transformers			[4]			3			
Base			Never Say Never			[3]			1			
SVM			Toy Story			[4]			4			
SVM			Avatar			[4]			4			
SVM			Titanic			[4]			3			
SVM			Transformers			[3]			3			
SVM			Never Say Never			[4]			1			
DT			Toy Story			[4]			4			
DT			Avatar			[4]			4			
DT			Titanic			[4]			3			
DT			Transformers			[3]			3			
DT			Never Say Never			[1]			1			
NB			Toy Story			[4]			4			
NB			Avatar			[3]			4			
NB			Titanic			[4]			3			
NB			Transformers			[3]			3			
NB			Never Say Never			[1]			1			
KNN			Toy Story			[2]			4			
KNN			Avatar			[4]			4			
KNN			Titanic			[4]			3			
KNN			Transformers			[4]			3			
KNN			Never Say Never			[3]			1			
RandomForest			Toy Story			[4]			4			
RandomForest			Avatar			[4]			4			
RandomForest			Titanic			[3]			3			
RandomForest			Transformers			[3]			3			
RandomForest			Never Say Never			[1]			1			



/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.

# Parte de los clasificadores - Version Antigua e ineficiente.

## KNN basico



In [68]:
# Primer Clasificador : K-nearnest-neighbors

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(movies, score)
score_pred = knn.predict(movies)
accuracy_score(score, score_pred)

print(classification_report(score,score_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        15
          2       0.51      0.29      0.37       459
          3       0.68      0.82      0.74      2618
          4       0.61      0.47      0.53      1352
          5       0.00      0.00      0.00         3

avg / total       0.64      0.65      0.63      4452



## KNN manual


In [69]:
# Primer Clasificador : K-nearnest-neighbors


X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.33)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))


             precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
          1       0.00      0.00      0.00         4
          2       0.24      0.24      0.24       140
          3       0.62      0.70      0.66       858
          4       0.50      0.39      0.44       465
          5       0.00      0.00      0.00         1

avg / total       0.55      0.56      0.55      1470



## KNN - Cross Validation

In [70]:
knn_cross = cross_validation.cross_val_score(knn, movies, score, cv=3)

print knn_cross
print("Accuracy: %0.2f (+/- %0.2f)" % (knn_cross.mean(), knn_cross.std() * 2))




[ 0.53131313  0.54410774  0.51147099]
Accuracy: 0.53 (+/- 0.03)


Mejora al momento de reducir las clases

# Support Vector Machine - SVM

In [71]:

movies_svm = svm.SVC()
movies_svm.fit(movies, score)  

score_pred = movies_svm.predict(movies)
accuracy_score(score, score_pred)

print(classification_report(score,score_pred))


             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        15
          2       0.00      0.00      0.00       459
          3       0.61      0.95      0.74      2618
          4       0.63      0.17      0.26      1352
          5       0.00      0.00      0.00         3

avg / total       0.55      0.61      0.52      4452



## SVM con Cross-Validation

In [74]:
svm_cross = svm.SVC()
cross = cross_validation.cross_val_score(svm_cross, movies, score, cv=10)

#Estadisticos respecto al cross validation
print cross
print("Accuracy: %0.2f (+/- %0.2f)" % (cross.mean(), cross.std() * 2))

#Res previos [ 0.61771058  0.60606061  0.60505415]
#Accuracy: 0.61 (+/- 0.01)

/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


[ 0.62276786  0.62053571  0.59284116  0.60538117  0.60089686  0.6036036
  0.59459459  0.6036036   0.61625282  0.61312217]
Accuracy: 0.61 (+/- 0.02)


## SVM test set manual

In [80]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.33, random_state=45)


svm_manual = svm.SVC()
svm_manual.fit(X_train, y_train)
y_pred = svm_manual.predict(X_test)
accuracy_score(y_test, y_pred)
# test_size, random_state
# 0,33 , 37 -> 0.59863945578231292
# 0,33 , 45 -> 0.61360544217687074
# 0,33 , 80 -> 0.60612244897959189

# 0,66 , 37 -> 0.60598843143926506
# 0,66 , 45 -> 0.60428717250765562


0.61360544217687074

# Random Forest

## Random forest con cross validation.

In [103]:
#Random forest con cross-validation.

forest_cross = RandomForestClassifier(n_estimators=10)
forest_cross_score = cross_validation.cross_val_score(forest_cross, movies, score, cv=3)
print forest_cross_score
print("Accuracy: %0.2f (+/- %0.2f)" % (forest_cross_score.mean(), forest_cross_score.std() * 2))


# cv = 3, n = 10
# array([ 0.54456929,  0.57389347,  0.54954955])

# cv = 3, n = 10
# array([ 0.55735661,  0.57927591,  0.59699625,  0.55319149,  0.57446809])



[ 0.54478114  0.56969697  0.5802969 ]
Accuracy: 0.56 (+/- 0.03)


## Random Forest test set Manual

In [82]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.3)
#, random_state=37

forest_manual = RandomForestClassifier(n_estimators=10)
forest_manual.fit(X_train, y_train)
y_pred = forest_manual.predict(X_test)
accuracy_score(y_test, y_pred)


# test_size = 0.33
# 0.57166666666666666

# test_size = 0.5
# 0.55449999999999999


0.57211538461538458

# Arbol de Decision

In [83]:
movies_tree = tree.DecisionTreeClassifier()
cross_validation.cross_val_score(movies_tree, movies, score, cv=10)


/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


array([ 0.54415274,  0.53349282,  0.51079137,  0.55395683,  0.51318945,
        0.5373494 ,  0.55903614,  0.56280193,  0.56763285,  0.5410628 ])

Aparentemente, tanto el arbol de decision como random forest tree no tienen un rendimiento mejor que los otros. ¿Por qué?

# Dummy Classifier

Que debemos hacer ahora ? -> Validar los modelos comparandolos con un dummy 